In [2]:
import json
from pycocotools.coco import COCO
import os
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer

In [68]:
def back_translation(text, first_model, first_model_tokenizer, second_model, second_model_tokenizer):
    
    fisrt_formed_text = f">>fr<< {text}"
    first_translated = first_model.generate(**first_model_tokenizer(fisrt_formed_text, return_tensors="pt", padding=True))
    first_translated_text = first_model_tokenizer.decode(first_translated[0], skip_special_tokens=True)
    second_formed_text = f">>en<< {first_translated_text}"
    second_translated = second_model.generate(**second_model_tokenizer(second_formed_text, return_tensors="pt", padding=True))
    first_translated_text = second_model_tokenizer.decode(second_translated[0], skip_special_tokens=True)
    
    return first_translated_text

In [69]:
first_model_name = 'Helsinki-NLP/opus-mt-en-fr'
first_model_tokenizer = MarianTokenizer.from_pretrained(first_model_name)
first_model = MarianMTModel.from_pretrained(first_model_name)
second_model_name = 'Helsinki-NLP/opus-mt-fr-en'
second_model_tokenizer = MarianTokenizer.from_pretrained(second_model_name)
second_model = MarianMTModel.from_pretrained(second_model_name)

In [72]:
import json
import pandas as pd

json_path = "/root/RobustMixGen_past/data/coco_train.json"

with open(json_path, 'r') as file:
    data = json.load(file)

json_df = pd.DataFrame(data)

In [94]:
def text_augmentation(obj_id, bg_id, json_df):
    """
    obj_id, bg_id 는 모두 'coco_image_id' 형식으로 삽입
    5개의 caption 중 첫번째 caption을 가져오는 코드로 우선 작성
    obj는 2번째 token 까지, bg는 3번째 token부터 끝까지를 가져오고,
    2개의 token을 concat하여 back translation을 진행
    이 부분은 추후 변경 필요
    """
    obj_caption = json_df[json_df['image_id'] == obj_id]['caption'].iloc[0]
    bg_caption = json_df[json_df['image_id'] == bg_id]['caption'].iloc[0]
    
    obj_tok = first_model_tokenizer.encode(obj_caption)
    bg_tok = first_model_tokenizer.encode(bg_caption)
    
    concat_token = obj_tok[:2] + bg_tok[2:]
    
    concat_text = first_model_tokenizer.decode(concat_token, skip_special_tokens=True)
    
    backtranslated_text = back_translation(concat_text, first_model, first_model_tokenizer, second_model, second_model_tokenizer)
    
    return backtranslated_text

In [95]:
test_text = text_augmentation("coco_522418", "coco_475546", json_df)

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (512) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [97]:
obj_caption = json_df[json_df['image_id'] == "coco_522418"]['caption'].iloc[0]
bg_caption = json_df[json_df['image_id'] == "coco_475546"]['caption'].iloc[0]

In [98]:
obj_caption , bg_caption

('A woman wearing a net on her head cutting a cake. ',
 'The patrons enjoy their beverages at the bar.')

In [96]:
test_text

'A woman takes advantage of her drinks at the bar.'